In [ ]:
x = input('What is your stock? ')

In [ ]:
print(x)

In [ ]:
import pandas as pd
import hvplot.pandas
import yfinance as yf
from dotenv import load_dotenv
import numpy as np
%matplotlib inline

In [ ]:
# Get historical data in YAHOO FINANCE HISTORICAL DATABASE
stock = yf.Ticker(x)

hist = stock.history(period="3y")

# Create an empty DataFrame for closing prices
signals_df = pd.DataFrame()

# Fetch the closing prices
signals_df[x] = hist["Close"]

signals_df.tail()

In [ ]:
# Fetch current X stock price
current_price = float(signals_df.iloc[-1])
actual = print(f" {x} current stock price is ${current_price}")

In [ ]:
# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df[x].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df[x].rolling(window=long_window).mean()

# Initialize the new `Signal` column
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] < signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Print the DataFrame
signals_df.tail()

In [ ]:
# Visualize exit position relative to close price
def signal(stock,color):
    stock_entry = stock.hvplot.scatter(
        color=color, legend=False, ylabel="Price in $", width=800, height=400, marker='v', size=200)
    return stock_entry


# Visualize close price for the investment
def close_price(stock,title):
    stock_close = stock.hvplot(
        line_color="lightgray", ylabel="Price in $", width=800, 
        title = title
)
    return stock_close

# Visualize moving averages
def averages(stock):
    moving_avgs = stock.hvplot(
        ylabel="Price in $", width=800, height=300
)
    return moving_avgs


In [ ]:
# Visualize entry position relative to close price
entry_signal = signal(
    stock = signals_df[signals_df["Entry/Exit"] == -1.0][x],
    color ="green")
# Visualize exit position relative to close price
exit_signal = signal(
    stock = signals_df[signals_df["Entry/Exit"] == 1.0][x],
    color ="red")

closing_prices = close_price(
    stock = signals_df[[x]],
    title="Stock Closing Price & Entry/Exit Signals")

# Visualize moving averages
moving =  averages(
    stock = signals_df[["SMA50", "SMA100"]])


In [ ]:
algo_chart = closing_prices * moving * entry_signal * exit_signal

In [ ]:
# Get historical data in YAHOO FINANCE HISTORICAL DATABASE
stock = yf.Ticker(x)

hist = stock.history(period="3y")

# Create an empty DataFrame for closing prices
features_df = pd.DataFrame()

# Fetch the closing prices
features_df = hist

features_df.tail()

In [ ]:
# Drop Dividends and Stock Splits columns
features_df = features_df.drop(columns=["Dividends", "Stock Splits"])
# Drop NAs and calculate daily percent return
features_df['daily_return'] = features_df['Close'].dropna().pct_change()
features_df

### Construct Exponential Moving Average of Closing Prices (Trading Signal, Feature One)

In [ ]:
 # Set short and long windows
short_window = 1
long_window = 10

# Construct a `Fast` and `Slow` Exponential Moving Average from short and long windows, respectively
features_df['fast_close'] = features_df['Close'].ewm(halflife=short_window).mean()
features_df['slow_close'] = features_df['Close'].ewm(halflife=long_window).mean()

# Construct a crossover trading signal
features_df['crossover_long'] = np.where(features_df['fast_close'] > features_df['slow_close'], 1.0, 0.0)
features_df['crossover_short'] = np.where(features_df['fast_close'] < features_df['slow_close'], -1.0, 0.0)
features_df['crossover_signal'] = features_df['crossover_long'] + features_df['crossover_short']

features_df.head()

In [ ]:
def ema(stock, title):
    feature_close = stock.hvplot.line(figsize=(20,10),
    title = title
)
    return feature_close

In [ ]:
ema_close = ema(
    stock = features_df[['Close', 'fast_close', 'slow_close']], 
    title = "Exponential Moving Average of Closing Prices")

### Construct Exponential Moving Average of Daily Return Volatility (Trading Signal, Feature Two)

In [ ]:
 # Set short and long volatility windows
short_vol_window = 1
long_vol_window = 10

# Construct a `Fast` and `Slow` Exponential Moving Average from short and long windows, respectively
features_df['fast_vol'] = features_df['daily_return'].ewm(halflife=short_vol_window).std()
features_df['slow_vol'] = features_df['daily_return'].ewm(halflife=long_vol_window).std()

# Construct a crossover trading signal
features_df['vol_trend_long'] = np.where(features_df['fast_vol'] < features_df['slow_vol'], 1.0, 0.0)
features_df['vol_trend_short'] = np.where(features_df['fast_vol'] > features_df['slow_vol'], -1.0, 0.0) 
features_df['vol_trend_signal'] = features_df['vol_trend_long'] + features_df['vol_trend_short']

features_df

In [ ]:
ema_vol = ema(
    stock = features_df[['fast_vol', 'slow_vol']], 
    title = "Exponential Moving Average of Daily Return Volatility")
ema_vol

### Construct Bollinger Band (Trading Signal, Feature Three)

In [ ]:
 # Set bollinger band window
bollinger_window = 20

# Calculate rolling mean and standard deviation
features_df['bollinger_mid_band'] = features_df['Close'].rolling(window=bollinger_window).mean()
features_df['bollinger_std'] = features_df['Close'].rolling(window=20).std()

# Calculate upper and lowers bands of bollinger band
features_df['bollinger_upper_band']  = features_df['bollinger_mid_band'] + (features_df['bollinger_std'] * 1)
features_df['bollinger_lower_band']  = features_df['bollinger_mid_band'] - (features_df['bollinger_std'] * 1)

# Calculate bollinger band trading signal
features_df['bollinger_long'] = np.where(features_df['Close'] < features_df['bollinger_lower_band'], 1.0, 0.0)
features_df['bollinger_short'] = np.where(features_df['Close'] > features_df['bollinger_upper_band'], -1.0, 0.0)
features_df['bollinger_signal'] = features_df['bollinger_long'] + features_df['bollinger_short']

features_df

In [ ]:
# Plot the Bollinger Bands for closing prices
bollinger = ema(features_df[['Close','bollinger_mid_band','bollinger_upper_band','bollinger_lower_band']],
              title = "Bollinger Band")

In [ ]:
import panel as pn

In [ ]:
# Create a Title for the Dashboard
dashboard_title = "Stock Analysis"

# Define Algo Trading Tab
algo_trading = pn.Column(
    "Algorithmic trading uses a computer program that follows a Criteria that shows when its 50-day moving average goes above/below the 100-day moving average, thereby identifies trends, then point in the graph BUY / SELL (Entry / Exit) recommendations",
        algo_chart)

# Define Trading Signal Features Tab
features_trading = pn.Column(
    "Trading Signal", 
        ema_close,
        ema_vol,
        bollinger)

In [ ]:
# Create a tab layout for the dashboard
dashboard = pn.Column(dashboard_title,
                      pn.Tabs(
                          ("Algo Trading", algo_trading),
                          ("Trading Signal Features", features_trading)
))

In [ ]:
dashboard.servable()